<a href="https://colab.research.google.com/github/harirm85/Hari_ML_workshop/blob/main/EDA1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.float_format = '{:,.2f}'.format

In [2]:
# Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Copy the Jason file and set security
!mkdir -p ~/.kaggle
!cp '/content/drive/MyDrive/Kaggle/kaggle.json' ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

#Download data
!kaggle competitions download -c home-credit-default-risk

import zipfile

# Specify the path to the downloaded zip file
zip_path = "/content/home-credit-default-risk.zip"

# Specify the directory to extract the files to
extract_path = "/content/home-credit-default-risk"

# Extract the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

Mounted at /content/drive
 98% 671M/688M [00:08<00:00, 83.6MB/s]
100% 688M/688M [00:08<00:00, 84.6MB/s]


In [3]:
#Take app train data and load to DF
app_data = pd.read_csv('/content/home-credit-default-risk/application_train.csv')
bur_data = pd.read_csv('/content/home-credit-default-risk/bureau.csv')

print(bur_data.shape)

(1716428, 17)


In [4]:
bur_data.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1716428 entries, 0 to 1716427
Data columns (total 17 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   SK_ID_CURR              1716428 non-null  int64  
 1   SK_ID_BUREAU            1716428 non-null  int64  
 2   CREDIT_ACTIVE           1716428 non-null  object 
 3   CREDIT_CURRENCY         1716428 non-null  object 
 4   DAYS_CREDIT             1716428 non-null  int64  
 5   CREDIT_DAY_OVERDUE      1716428 non-null  int64  
 6   DAYS_CREDIT_ENDDATE     1610875 non-null  float64
 7   DAYS_ENDDATE_FACT       1082775 non-null  float64
 8   AMT_CREDIT_MAX_OVERDUE  591940 non-null   float64
 9   CNT_CREDIT_PROLONG      1716428 non-null  int64  
 10  AMT_CREDIT_SUM          1716415 non-null  float64
 11  AMT_CREDIT_SUM_DEBT     1458759 non-null  float64
 12  AMT_CREDIT_SUM_LIMIT    1124648 non-null  float64
 13  AMT_CREDIT_SUM_OVERDUE  1716428 non-null  float64
 14  CR

In [27]:
bur_data[bur_data['AMT_CREDIT_SUM_OVERDUE']>0].head(2)

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
365,282297,5714943,Active,currency 1,-2712,2603,"-2,498.00",NaN,"4,590.00",0,"24,696.00",231.53,0.00,231.53,Consumer credit,-1077,NaN
785,408647,5715468,Active,currency 1,-313,6,233.00,NaN,NaN,0,"417,208.50","214,947.00",0.00,288.00,Consumer credit,-5,NaN


In [42]:
bur_data['DAYS_CREDIT_UPDATE'].value_counts()

-7        18503
-8        18462
-11       16975
-15       16870
-12       16827
          ...  
-41856        1
-41940        1
-2894         1
-2913         1
-2844         1
Name: DAYS_CREDIT_UPDATE, Length: 2982, dtype: int64

In [59]:
from pandas.core.dtypes.cast import Sized
#bur_data1 = bur_data[bur_data['SK_ID_CURR']==282297]

Summary_data = bur_data.groupby('SK_ID_CURR', as_index=False).agg(
    BUR_CNT = ("SK_ID_BUREAU","count")
    , BUR_ACTIVE_CNT = ("CREDIT_ACTIVE", lambda x: sum(x=='Active'))
    , BUR_SOLD_CNT = ("CREDIT_ACTIVE", lambda x: sum(x=='Sold'))
    , BUR_BAD_DEPT_CNT = ("CREDIT_ACTIVE", lambda x: sum(x=='Bad debt'))
    , BUR_CUR_USED = ("CREDIT_CURRENCY","nunique")
    , BUR_CRD_TYPES = ("CREDIT_TYPE","nunique")
    , BUR_CRD_TYPES_OTHR=("CREDIT_TYPE", lambda x: sum(x.isin(['Consumer credit', 'Credit card'
                                                  , 'Car loan', 'Mortgage']) == False))
    , BUR_DAYS_CRD_VAR = ("DAYS_CREDIT", lambda x:max(x) - min(x))
    , BUR_DAYS_CRD_MIN = ("DAYS_CREDIT", "max")
    , BUR_OD_DAY_CNT = ("CREDIT_DAY_OVERDUE", lambda x:sum(x>0))
    , BUR_OD_DAY_VAR = ("CREDIT_DAY_OVERDUE", lambda x:max(x) - min(x))
    , BUR_OD_DAY_MIN = ("CREDIT_DAY_OVERDUE", "min")
    , BUR_DAYS_CRD_ED_CNT1 = ("DAYS_CREDIT_ENDDATE", lambda x:sum(x>0))
    , BUR_DAYS_CRD_ED_CNT2 = ("DAYS_CREDIT_ENDDATE", lambda x:sum(x<0))
    , BUR_DAYS_ED_FACT_CNT = ("DAYS_ENDDATE_FACT", lambda x:sum(x>0))
    , BUR_DAYS_ED_FACT_VAR = ("DAYS_ENDDATE_FACT", lambda x:max(x) - min(x))
    , BUR_OD_AMT_MAX_CNT = ("AMT_CREDIT_MAX_OVERDUE", lambda x:sum(x>0))
    , BUR_OD_AMT_MAX_VAR = ("AMT_CREDIT_MAX_OVERDUE", lambda x:max(x) - min(x))
    , BUR_CRD_PRO_CNT = ("CNT_CREDIT_PROLONG", lambda x:sum(x>0))
    , BUR_CRD_PRO_VAR = ("CNT_CREDIT_PROLONG", lambda x:max(x) - min(x))
    , BUR_CRD_AMT_VAR = ("AMT_CREDIT_SUM", lambda x:max(x) - min(x))
    , BUR_CRD_DBT_CNT = ("AMT_CREDIT_SUM_DEBT", lambda x:sum(x>0))
    , BUR_CRD_DBT_MAX = ("AMT_CREDIT_SUM_DEBT", "max")
    , BUR_CRD_LMT_CNT = ("AMT_CREDIT_SUM_LIMIT", lambda x:sum(x>0))
    , BUR_CRD_LMT_MAX = ("AMT_CREDIT_SUM_LIMIT", "max")
    , BUR_OD_AMT_SUM_CNT = ("AMT_CREDIT_SUM_OVERDUE", lambda x:sum(x>0))
    , BUR_OD_AMT_SUM_VAR = ("AMT_CREDIT_SUM_OVERDUE", lambda x:max(x) - min(x))
    , BUR_DAYS_CRD_UPDT_VAR = ("DAYS_CREDIT_UPDATE", lambda x:max(x) - min(x))
    , BUR_DAYS_CRD_UPDT_MIN = ("DAYS_CREDIT_UPDATE", "max")
    , BUR_ANNUITY_VAR = ("AMT_CREDIT_SUM_DEBT", lambda x:max(x) - min(x))
)
Summary_data.shape

(305811, 31)

In [63]:
Summary_data[Summary_data['BUR_OD_AMT_SUM_VAR']>0][['BUR_CRD_AMT_VAR','BUR_CRD_DBT_MAX','BUR_CRD_LMT_MAX','BUR_OD_AMT_SUM_VAR','BUR_OD_AMT_MAX_VAR']]

,BUR_CRD_AMT_VAR,BUR_CRD_DBT_MAX,BUR_CRD_LMT_MAX,BUR_OD_AMT_SUM_VAR,BUR_OD_AMT_MAX_VAR
103,"143,910.00","108,837.00",0.00,166.50,NaN
139,"961,384.50","850,653.00",0.00,13.50,"16,320.51"
298,"287,874.54","204,894.00",0.00,459.00,"29,654.01"
306,"886,500.00","430,722.00",0.00,"19,359.00",NaN
475,"166,500.00","175,464.00",0.00,360.00,NaN
...,...,...,...,...,...
305464,"1,398,496.50","937,233.00","50,700.64",522.00,"5,490.00"
305470,"844,024.50","584,181.00",0.00,477.00,NaN
305580,"33,997.50",0.00,0.00,9.00,NaN
305687,"112,500.00","69,404.67",0.00,13.50,NaN


In [80]:
Summary_data['BUR_MAX_OD_PER_OD'] = Summary_data['BUR_OD_AMT_MAX_VAR']/Summary_data['BUR_OD_AMT_SUM_VAR']
#Summary_data['RTO2'] = Summary_data['BUR_CRD_LMT_MAX']/Summary_data['BUR_CRD_DBT_MAX']

In [86]:
combined_data=app_data[['SK_ID_CURR','TARGET']].join(Summary_data,on='SK_ID_CURR',lsuffix='_APP', rsuffix='_BUR')
combined_data.shape

(307511, 34)

In [87]:
combined_data.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Data columns (total 34 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   SK_ID_CURR_APP         307511 non-null  int64  
 1   TARGET                 307511 non-null  int64  
 2   SK_ID_CURR_BUR         177604 non-null  float64
 3   BUR_CNT                177604 non-null  float64
 4   BUR_ACTIVE_CNT         177604 non-null  float64
 5   BUR_SOLD_CNT           177604 non-null  float64
 6   BUR_BAD_DEPT_CNT       177604 non-null  float64
 7   BUR_CUR_USED           177604 non-null  float64
 8   BUR_CRD_TYPES          177604 non-null  float64
 9   BUR_CRD_TYPES_OTHR     177604 non-null  float64
 10  BUR_DAYS_CRD_VAR       177604 non-null  float64
 11  BUR_DAYS_CRD_MIN       177604 non-null  float64
 12  BUR_OD_DAY_CNT         177604 non-null  float64
 13  BUR_OD_DAY_VAR         177604 non-null  float64
 14  BUR_OD_DAY_MIN         177604 non-nu

In [88]:
# correlation with Target - target value
corr_features = pd.DataFrame()
corr_features['FI'] = combined_data.corr()['TARGET']
corr_features['FI_MOD']  = corr_features['FI'].abs()
print(corr_features.sort_values(by='FI_MOD',ascending=False).head(50)) 
#corr_features.style.background_gradient(cmap='viridis')

                         FI  FI_MOD
TARGET                 1.00    1.00
BUR_MAX_OD_PER_OD     -0.03    0.03
BUR_DAYS_CRD_UPDT_VAR -0.01    0.01
BUR_DAYS_CRD_VAR      -0.01    0.01
BUR_DAYS_CRD_MIN      -0.01    0.01
BUR_DAYS_CRD_ED_CNT1  -0.01    0.01
BUR_DAYS_CRD_UPDT_MIN -0.01    0.01
BUR_CNT               -0.01    0.01
BUR_ACTIVE_CNT        -0.01    0.01
BUR_CRD_LMT_CNT       -0.01    0.01
BUR_CRD_AMT_VAR       -0.01    0.01
BUR_CRD_DBT_MAX       -0.01    0.01
BUR_ANNUITY_VAR       -0.00    0.00
BUR_DAYS_ED_FACT_VAR  -0.00    0.00
BUR_CRD_DBT_CNT       -0.00    0.00
BUR_CRD_TYPES         -0.00    0.00
BUR_OD_DAY_CNT         0.00    0.00
BUR_DAYS_CRD_ED_CNT2  -0.00    0.00
BUR_SOLD_CNT          -0.00    0.00
BUR_OD_AMT_MAX_CNT    -0.00    0.00
BUR_OD_AMT_SUM_VAR    -0.00    0.00
BUR_OD_AMT_SUM_CNT     0.00    0.00
BUR_OD_AMT_MAX_VAR    -0.00    0.00
BUR_CRD_PRO_VAR        0.00    0.00
BUR_OD_DAY_VAR         0.00    0.00
BUR_CRD_LMT_MAX       -0.00    0.00
SK_ID_CURR_APP        -0.00 